In [ ]:
# !pip install accelerate peft bitsandbytes transformers trl

In [1]:
import os
num_cores = os.cpu_count()
print(f"Number of CPU cores available: {num_cores}")


Number of CPU cores available: 12


In [2]:
import torch

# Set the number of CPU threads PyTorch can use
num_threads = 6  # or you can set this to a specific number less than num_cores
torch.set_num_threads(num_threads)

print(f"Number of threads PyTorch will use: {torch.get_num_threads()}")


Number of threads PyTorch will use: 6


In [3]:
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [4]:
dataset="burkelibbey/colors"
model_id="TinyLlama/TinyLlama-1.1B-Chat-v1.0" # using somewhat fine tuned version of the model
output_model="tinyllama-colorist-v1"

### Data preparation

In [8]:
# we need to reformat the data in teh ChatML format.

def formatted_train(input,response)->str:
    return f"<|user|>\n{input}</s>\n<|assistant|>\n{response}</s>"

In [9]:
def prepare_train_data(data_id):
    data = load_dataset(data_id, split="train")
    data_df = data.to_pandas()
    data_df["text"] = data_df[["description", "color"]].apply(lambda x: "<|user|>\n" + x["description"] + "</s>\n<|assistant|>\n" + x["color"] + "</s>", axis=1)
    data = Dataset.from_pandas(data_df)
    return data

In [10]:
data = prepare_train_data(dataset)

In [11]:
data

Dataset({
    features: ['color', 'description', 'text'],
    num_rows: 33887
})

In [12]:
data[0]

{'color': '#000000',
 'description': 'Pure Black: A shade that completely absorbs light and does not reflect any colors. It is the darkest possible shade.',
 'text': '<|user|>\nPure Black: A shade that completely absorbs light and does not reflect any colors. It is the darkest possible shade.</s>\n<|assistant|>\n#000000</s>'}

### Model the Model (not the base version)

In [13]:
def get_model_and_tokenizer(model_id):

    tokenizer = AutoTokenizer.from_pretrained(model_id) # Tokenizers are in charge of preparing inputs for a model
    tokenizer.pad_token = tokenizer.eos_token
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )
    model = AutoModelForCausalLM.from_pretrained(
        model_id, quantization_config=bnb_config, device_map="auto"
    )
    model.config.use_cache=False
    model.config.pretraining_tp=1
    return model, tokenizer

In [14]:
model, tokenizer = get_model_and_tokenizer(model_id)

### Setting up the LoRA

In [15]:
peft_config = LoraConfig(
        r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )

In [16]:
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=2,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=3,
        max_steps=250,
        fp16=True,
        # push_to_hub=True
    )

In [17]:
trainer = SFTTrainer(
        model=model,
        train_dataset=data,
        peft_config=peft_config,
        dataset_text_field="text",
        args=training_arguments,
        tokenizer=tokenizer,
        packing=False,
        max_seq_length=1024
    )

C:\Users\vivek\anaconda3\envs\llm\lib\site-packages\huggingface_hub\utils\_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
C:\Users\vivek\anaconda3\envs\llm\lib\site-packages\trl\trainer\sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
C:\Users\vivek\anaconda3\envs\llm\lib\site-packages\trl\trainer\sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/33887 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [18]:
trainer.train()

C:\Users\vivek\anaconda3\envs\llm\lib\site-packages\transformers\models\llama\modeling_llama.py:671: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss
10,2.164600
20,1.839000
30,1.558400
40,1.397700
50,1.358900
60,1.264000
70,1.271100
80,1.280500
90,1.250300
100,1.251500


TrainOutput(global_step=250, training_loss=1.3081112861633302, metrics={'train_runtime': 2567.5891, 'train_samples_per_second': 0.779, 'train_steps_per_second': 0.097, 'total_flos': 950020454449152.0, 'train_loss': 1.3081112861633302, 'epoch': 0.059017941454202076})

### Merging the LoRA with the base model

In [20]:
from peft import AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM
import torch
import os

model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, load_in_8bit=False,
                                             device_map="auto",
                                             trust_remote_code=True,
                                            offload_buffers = True)

# model_path = "/content/tinyllama-colorist-v1/checkpoint-250"
model_path = "D:/tiny llama/tinyllama-colorist-v1/checkpoint-250"

peft_model = PeftModel.from_pretrained(model, model_path, from_transformers=True, device_map="auto")

model = peft_model.merge_and_unload()

C:\Users\vivek\anaconda3\envs\llm\lib\site-packages\accelerate\utils\modeling.py:1363: UserWarning: Current model requires 11537152 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


In [21]:
model


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

### Inference from the LLM

In [22]:
from transformers import GenerationConfig
from time import perf_counter

def generate_response(user_input):

  prompt = formatted_prompt(user_input)

  inputs = tokenizer([prompt], return_tensors="pt")
  generation_config = GenerationConfig(penalty_alpha=0.6,do_sample = True,
      top_k=5,temperature=0.5,repetition_penalty=1.2,
      max_new_tokens=12,pad_token_id=tokenizer.eos_token_id
  )
  start_time = perf_counter()

  inputs = tokenizer(prompt, return_tensors="pt").to('cuda')

  outputs = model.generate(**inputs, generation_config=generation_config)
  print(tokenizer.decode(outputs[0], skip_special_tokens=True))
  output_time = perf_counter() - start_time
  print(f"Time taken for inference: {round(output_time,2)} seconds")

In [23]:
def formatted_prompt(question)-> str:
    return f"<|user|>\n{question}</s>\n<|assistant|>"

In [24]:
def print_color_space(hex_color):
    def hex_to_rgb(hex_color):
        hex_color = hex_color.lstrip('#')
        return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    r, g, b = hex_to_rgb(hex_color)
    print(f'{hex_color}: \033[48;2;{r};{g};{b}m           \033[0m')

In [31]:
generate_response(user_input='royal blue')

<|user|>
royal blue 
<|assistant|>
#241080 // This is a
Time taken for inference: 7.03 seconds


In [26]:
generate_response(user_input='A dull baby pink')

<|user|>
A dull baby pink 
<|assistant|>
#b08866 // This is a
Time taken for inference: 6.71 seconds


In [32]:
import torch

# Save the model's state_dict
torch.save(model.state_dict(), 'model.pth')


In [35]:
torch.save({
    'epoch': 3,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': paged_adamw_32bit,
    'loss': loss,
}, 'model_checkpoint.pth')


NameError: name 'paged_adamw_32bit' is not defined

In [37]:
# Save the model, tokenizer, and training arguments
output_model = "D:/tiny llama/saved model"
trainer.save_model(output_model)
